In [1]:
# Import needed libraries
import pandas as pd
import numpy as np
import re

In [2]:
# Import files employees

df_employees = pd.read_csv("employees.csv",sep=',')
df_employees = df_employees.drop_duplicates(subset=['employe_id'],keep='last')
df_employees.rename(columns={'employe_id': 'employee_id'}, inplace=True)

df_employees.head()

,employee_id,branch_id,salary,join_date,resign_date
0,1,3,7500000,2018-08-23,NaN
1,7,1,7500000,2017-04-28,NaN
2,8,1,13000000,2017-04-28,NaN
3,9,1,13500000,2017-12-22,2020-10-14
4,10,1,9000000,2018-03-01,2020-12-20


In [3]:
# Import files timesheet
df_timesheets = pd.read_csv("timesheets.csv",sep=',')
df_timesheets = df_timesheets.drop_duplicates(subset=['timesheet_id'],keep='last')
df_timesheets.head()

,timesheet_id,employee_id,date,checkin,checkout
0,23907432,66,2019-08-21,08:13:31,17:05:02
1,23907433,22,2019-08-21,08:56:34,18:00:48
2,23907434,21,2019-08-21,09:45:08,18:24:06
3,23907435,63,2019-08-21,09:55:47,NaN
4,23907437,60,2019-08-21,09:56:05,17:31:08


In [4]:
# transformation timesheet
# get time difference

df_timesheets['time_diff'] = pd.to_timedelta(df_timesheets['checkout']) - pd.to_timedelta(df_timesheets['checkin'])
df_timesheets['time_diff'] = df_timesheets['time_diff'].replace(np.nan,pd.to_timedelta('07:00:00'))

# Change negative timedelta values to '07:00:00'
replacement_timedelta = pd.to_timedelta('07:00:00')
df_timesheets['time_diff'] = df_timesheets['time_diff'].apply(lambda td: replacement_timedelta if td < pd.Timedelta(0) else td)

df_timesheets

,timesheet_id,employee_id,date,checkin,checkout,time_diff
0,23907432,66,2019-08-21,08:13:31,17:05:02,0 days 08:51:31
1,23907433,22,2019-08-21,08:56:34,18:00:48,0 days 09:04:14
2,23907434,21,2019-08-21,09:45:08,18:24:06,0 days 08:38:58
3,23907435,63,2019-08-21,09:55:47,NaN,0 days 07:00:00
4,23907437,60,2019-08-21,09:56:05,17:31:08,0 days 07:35:03
...,...,...,...,...,...,...
39709,70649593,59,2020-12-23,08:05:29,17:05:18,0 days 08:59:49
39710,70649597,109100,2020-12-23,08:05:30,17:05:30,0 days 09:00:00
39711,70650186,9908,2020-12-23,08:06:52,NaN,0 days 07:00:00
39712,70650458,70,2020-12-23,08:07:33,17:13:27,0 days 09:05:54


In [5]:
# join table timesheets and employees
join_employee = pd.merge(df_employees, df_timesheets, on='employee_id', how='inner')
join_employee['year'] = pd.to_datetime(join_employee['date']).dt.year
join_employee['month'] = pd.to_datetime(join_employee['date']).dt.month

In [6]:
# Aggregation table to get unique salary per employee id and total working hours
summary = join_employee[['year','month','branch_id','employee_id','salary','time_diff']]
summary = summary.groupby(['year', 'month','branch_id','employee_id']).agg({'salary': 'max', 'time_diff': 'sum'}).reset_index()
summary['time_diff'] = summary['time_diff'].dt.total_seconds() / 3600
summary

,year,month,branch_id,employee_id,salary,time_diff
0,2019,8,1,21,13500000,59.115556
1,2019,8,1,22,8000000,59.558889
2,2019,8,2,60,12500000,51.767222
3,2019,8,2,63,13500000,51.845278
4,2019,8,2,66,10000000,64.394722
...,...,...,...,...,...,...
2004,2020,12,3092,26648,13000000,121.647778
2005,2020,12,3092,33936,7500000,89.257222
2006,2020,12,3092,49616,10000000,28.000000
2007,2020,12,11265,251119,11000000,151.282778


In [7]:
# Aggregating to get detail salary per hour
result_data = summary.groupby(['year', 'month','branch_id']).agg({'salary': 'sum', 'time_diff': 'sum'}).reset_index()
result_data['salary_per_hour'] = result_data['salary'] / result_data['time_diff']

final_data = result_data[['year','month','branch_id','salary_per_hour']]
final_data

,year,month,branch_id,salary_per_hour
0,2019,8,1,181167.900980
1,2019,8,2,214276.502664
2,2019,8,3,928571.428571
3,2019,9,1,173905.385437
4,2019,9,2,121069.414207
...,...,...,...,...
234,2020,12,2633,176546.745655
235,2020,12,2635,113420.683808
236,2020,12,3092,128712.052915
237,2020,12,11265,72711.515227


In [8]:
# Load DataFrame to CSV
final_data.to_csv('result_salary_per_hour.csv', index=False, sep= '|')  # Set index=False to not write row indices to the CSV
print("Data succesfully written to 'result_salary_per_hour.csv'")

Data succesfully written to 'result_salary_per_hour.csv'
